In [1]:
import random
import string
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import json

def generate_password(length=12, include_letters=True, include_numbers=True, include_symbols=True):
    characters = ""
    if include_letters:
        characters += string.ascii_letters
    if include_numbers:
        characters += string.digits
    if include_symbols:
        characters += string.punctuation

    if not characters:
        messagebox.showwarning("Invalid Configuration", "You must select at least one character type.")
        return ""

    password = ''.join(random.choice(characters) for _ in range(length))
    return password

def save_password():
    app_name = app_name_entry.get()
    username = username_entry.get()
    password_length = int(password_length_entry.get())
    include_letters = include_letters_var.get()
    include_numbers = include_numbers_var.get()
    include_symbols = include_symbols_var.get()

    generated_password = generate_password(password_length, include_letters, include_numbers, include_symbols)
    if generated_password:
        password_data[app_name] = {"username": username, "password": generated_password}

        with open("passwords.json", "w") as f:
            json.dump(password_data, f, indent=4)

        messagebox.showinfo("Password Saved", f"Password for {app_name} saved!")
        update_table()

def delete_password():
    selected_item = password_table.focus()
    if selected_item:
        app_name = password_table.item(selected_item, "text")
        if messagebox.askyesno("Confirm Deletion", f"Do you want to delete the password for '{app_name}'?"):
            password_data.pop(app_name, None)
            with open("passwords.json", "w") as f:
                json.dump(password_data, f, indent=4)
            update_table()
            messagebox.showinfo("Password Deleted", f"Password for '{app_name}' deleted.")

def admin_view_password():
    admin_password = simpledialog.askstring("Admin Authentication", "Enter Admin Password:", show='*')
    if admin_password == "roshan3100":  # Replace with your actual admin password
        selected_item = password_table.focus()
        if selected_item:
            app_name = password_table.item(selected_item, "text")
            password = password_data.get(app_name, {}).get("password")
            if password:
                password_masked = "*" * len(password)
                messagebox.showinfo(f"Password for {app_name}", f"Masked Password: {password_masked}\nActual Password: {password}")
            else:
                messagebox.showwarning("Password Not Found", f"No password found for '{app_name}'.")
        else:
            messagebox.showwarning("Selection Missing", "Please select an application from the table.")
    else:
        messagebox.showerror("Authentication Failed", "Admin password is incorrect.")

def update_table():
    for row in password_table.get_children():
        password_table.delete(row)
    
    for app_name, data in password_data.items():
        password_table.insert("", "end", text=app_name, values=(data["username"], "*" * len(data["password"])))

password_data = {}

try:
    with open("passwords.json", "r") as f:
        password_data = json.load(f)
except FileNotFoundError:
    pass

app = tk.Tk()
app.title("Password Manager")

tk.Label(app, text="App Name:").grid(row=0, column=0)
app_name_entry = tk.Entry(app)
app_name_entry.grid(row=0, column=1)

tk.Label(app, text="Username:").grid(row=1, column=0)
username_entry = tk.Entry(app)
username_entry.grid(row=1, column=1)

tk.Label(app, text="Password Length:").grid(row=2, column=0)
password_length_entry = tk.Entry(app)
password_length_entry.grid(row=2, column=1)

include_letters_var = tk.BooleanVar()
include_numbers_var = tk.BooleanVar()
include_symbols_var = tk.BooleanVar()

tk.Checkbutton(app, text="Include Letters", variable=include_letters_var).grid(row=3, column=0)
tk.Checkbutton(app, text="Include Numbers", variable=include_numbers_var).grid(row=3, column=1)
tk.Checkbutton(app, text="Include Symbols", variable=include_symbols_var).grid(row=3, column=2)

tk.Button(app, text="Generate and Save Password", command=save_password).grid(row=4, column=0, columnspan=3)
tk.Button(app, text="Delete Password", command=delete_password).grid(row=5, column=0, columnspan=3)
tk.Button(app, text="Admin View Password", command=admin_view_password).grid(row=6, column=0, columnspan=3)

# Creating the password table
password_table = ttk.Treeview(app, columns=("Username", "Password"))
password_table.heading("#1", text="Application")
password_table.heading("Username", text="Username")
password_table.heading("Password", text="Password")
password_table.column("#1", width=200)
password_table.column("Username", width=150)
password_table.column("Password", width=150)
password_table.grid(row=7, column=0, columnspan=3, padx=10, pady=10)

update_table()

app.mainloop()
